In [ ]:
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install transformers
!pip install torch
!pip install librosa
!pip install sklearn
!pip install pydub

#ffmpeg is a dependency for pydub. Can install at the system level with:
# brew install ffmpeg

### Directory Management to work with Audio Data files

In [ ]:
# combine the zip files into single zip file
!cat mp3.zip.* > single_mp3.zip

# unzip the mp3 files
!unzip mp3.zip

#move all mp3 files into mp3 folder
!find . -type f -name "*.mp3" -exec mv {} ./mp3/ \;

#remove all zip files
! rm -r 0 1 2 3 4 5 6 7 8 9 a b c d e f

### Converting Audio in MP3 to WAV

In [ ]:
#Converting mp3 files to wav files

import os
from pydub import AudioSegment

# Directory where mp3 files are stored
mp3_dir = './mp3'

#Directory where wav files will be saved
wav_dir = './wav'

#Checking to see if wav directory exists
os.makedirs(wav_dir, exist_ok=True)

#counter to keep track of audio files
processed = 0

#Loop through all mp3 files in mp3 directory
for mp3 in os.listdir(mp3_dir):
    if processed >= 26000:
        break #stop processing files after the number above has been processed
    
    if mp3.endswith('.mp3'):
        mp3_path = os.path.join(mp3_dir, mp3)
        try:
            #Loading mp3 file
            audio = AudioSegment.from_mp3(mp3_path)
            
            #Create the output wav file path by changing the extension from mp3 to wav
            wav_file = mp3.replace('.mp3', '.wav')
            wav_path = os.path.join(wav_dir, wav_file)

            #Export audio as wav file
            audio.export(wav_path, format='wav')
            print(f'Converted: {mp3} -> {wav_file}')
            
            #Increment counter
            processed += 1
        except Exception as e:
            print(f'Error converting: {mp3}: {str(e)}')

### Checking the distribution of the label data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

annotations = pd.read_csv('./annotations_final.csv', sep='\t')
clip_data = pd.read_csv('./clip_info_final.csv', sep= '\t')

#Check shape of data

# !find ./wav/ -type f | wc -l 
#print(annotations.shape)
# print(clip_data.shape)

#for each column, count the number of 1s and store in a dictionary

annotations_dict = {}
for column in annotations.columns:
    annotations_dict[column] = annotations[column].sum()
    
#remove first and last column    
del annotations_dict['clip_id'], annotations_dict['mp3_path']

#visualize distribution of variables in annotations
plt.bar(annotations_dict.keys(), annotations_dict.values())

#make the plot bigger
plt.rcParams['figure.figsize'] = [50, 10]

#rotate x axis labels
plt.xticks(rotation=90)
#make font size bigger
plt.rcParams.update({'font.size': 21})
#change font 
plt.rcParams['font.family'] = 'sans-serif'
plt.show()


### Make sure paths for annotations and text metadata match with audio directory. Also that the file types are correct.

In [ ]:
def clean_path(file):
    '''
    To clean the path of the file to merge the datasets
    '''
    file['mp3_path'] = file['mp3_path'] = 'mp3/' + file['mp3_path'].str.split('/').str[1] #replace the path with the correct path
    file['mp3_path'] = file['mp3_path'].str.replace('.mp3', '.wav') #change the file type to .wav

    #print the first few rows to check
    print(file['mp3_path'].head(3))
    
    return file

clip_data_clean = clean_path(clip_data)
annotations_clean = clean_path(annotations)

#output the new clip_data_clean into csv to inspect

clip_data_clean.to_csv('clip_data_clean.csv', index=False)
annotations_clean.to_csv('annotations_clean.csv', index=False)